<a href="https://colab.research.google.com/github/dudaholandah/NLP/blob/main/Projeto/Text_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r '/content/drive/MyDrive/Datasets/cuisine-classification-ingredients' 'cuisine-classification-ingredients'

In [3]:
!cp -r '/content/drive/MyDrive/Datasets/indian-food' 'indian-food'

# Imports

In [4]:
# !pip install -q -U "tensorflow-text==2.8.*"
# !pip install -q -U tf-models-official==2.7.0
# !pip install -U tfds-nightly
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 4.7 MB 34.0 MB/s 
     |████████████████████████████████| 1.3 MB 50.4 MB/s 
     |████████████████████████████████| 120 kB 47.2 MB/s 
     |████████████████████████████████| 6.6 MB 37.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=febf9ddb91ebbecd9b1f8d2d41e50baf1173d31d4cc2f34b83c4570746ef2289
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [5]:
#TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
#WORD2VEC
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
#BERT
import tensorflow as tf
from sentence_transformers import SentenceTransformer
# import tensorflow_hub as hub
# import tensorflow_datasets as tfds
# import tensorflow_text as text
# import tensorflow_addons as tfa
# from official.nlp import optimization
# tf.get_logger().setLevel('ERROR')

#Data
import json
import os
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Data

Cuisine Classification Data

In [6]:
path = 'cuisine-classification-ingredients/train.json'

with open(path, 'r') as f:
  json = json.load(f)

In [7]:
df_cuisine = pd.DataFrame(json, columns=['id', 'cuisine', 'ingredients'])

In [8]:
df_cuisine = df_cuisine.drop(['id'], axis=1)

In [9]:
df_cuisine.head(5)

,cuisine,ingredients
0,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,"[water, vegetable oil, wheat, salt]"
4,indian,"[black pepper, shallots, cornflour, cayenne pe..."


Indian Food Data

In [10]:
df_indian = pd.read_csv('indian-food/indian_food.csv')

In [11]:
df_indian.head(5)

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East


## Pre-Processing

In [12]:
def pre_processing(text):
  new = ""
  for word in text.split(','):
    word = re.sub(r'[^\w\s]', '', word.lower())
    word = re.sub(r'[0-9]+', '', word)
    word = re.sub(r'\boz\b', '', word)
    word = re.sub(r'\ba taste of\b', '', word)
    new = new + word.strip() + " "

  return new[:-1]

Cuisine Classification Data

In [13]:
df_cuisine['ingredients'] = [','.join(x).strip() for x in df_cuisine['ingredients']]

In [14]:
df_cuisine['ingredients'] = [pre_processing(x) for x in df_cuisine['ingredients']]

In [15]:
df_cuisine.head(5)

,cuisine,ingredients
0,greek,romaine lettuce black olives grape tomatoes ga...
1,southern_us,plain flour ground pepper salt tomatoes ground...
2,filipino,eggs pepper salt mayonaise cooking oil green c...
3,indian,water vegetable oil wheat salt
4,indian,black pepper shallots cornflour cayenne pepper...


Indian Food Data

In [16]:
df_indian['ingredients'] = [pre_processing(x) for x in df_indian['ingredients']]

In [17]:
df_indian.head(5)

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,maida flour yogurt oil sugar,vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,gram flour ghee sugar,vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,carrots milk sugar ghee cashews raisins,vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,flour ghee kewra milk clarified butter sugar a...,vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,milk powder plain flour baking powder ghee mil...,vegetarian,15,40,sweet,dessert,West Bengal,East


# Different Embeddings

## TFIDF

Cuisine Classification Data

In [18]:
tfidf_vectorizer = TfidfVectorizer()

In [19]:
tfidf_cuisine = tfidf_vectorizer.fit_transform(df_cuisine['ingredients'])

In [20]:
df_tfidf_cuisine = pd.DataFrame(tfidf_cuisine.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

In [21]:
df_tfidf_cuisine.head(5)

,abalone,abbamele,absinthe,abura,acai,accent,accompaniment,achiote,acid,acini,...,yuzukosho,zaatar,zatarains,zero,zest,zesty,zinfandel,ziti,zucchini,épices
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Indian Food Data

In [22]:
tfidf_indian = tfidf_vectorizer.fit_transform(df_indian['ingredients'])

In [23]:
df_tfidf_indian = pd.DataFrame(tfidf_indian.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

In [24]:
df_tfidf_indian.head(5)

,acid,all,almond,almonds,aloo,alum,amaranth,amchur,and,anise,...,whipping,white,whole,wild,wine,yam,yellow,yoghurt,yogurt,yolks
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.540083,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.349811,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


## Word2Vec

In [25]:
def get_embedding_w2v(model, doc_tokens):
  embeddings = []
  for tok in doc_tokens:
    if tok in model.wv.vocab:
      embeddings.append(model.wv.word_vec(tok))
    else:
      embeddings.append(np.random.rand(300))
  return np.mean(embeddings, axis=0)

Cuisine Classification Data

In [26]:
ing_cuisine = '. '.join(df_cuisine['ingredients'])
word2vec_cuisine = nltk.sent_tokenize(ing_cuisine)
word2vec_cuisine = [nltk.word_tokenize(sentence) for sentence in word2vec_cuisine]

In [27]:
model_word2vec = Word2Vec(word2vec_cuisine, size=300, min_count=1, window=5, sg=1,workers=4)

In [28]:
word2vec_cuisine = df_cuisine['ingredients'].apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))

Indian Food Data

In [29]:
ing_indian = '. '.join(df_indian['ingredients'])
word2vec_indian = nltk.sent_tokenize(ing_indian)
word2vec_indian = [nltk.word_tokenize(sentence) for sentence in word2vec_indian]

In [30]:
model_word2vec = Word2Vec(word2vec_indian, size=300, min_count=1, window=5, sg=1,workers=1)

In [31]:
word2vec_indian = df_indian['ingredients'].apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))

## BERT

In [32]:
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [33]:
bert_cuisine = bert_model.encode(df_cuisine['ingredients'])

In [34]:
bert_indian = bert_model.encode(df_indian['ingredients'])

# Different Classificators

# Text Retrieval

In [35]:
def cosine_similarity(vetor1,vetor2):

  prod_interno = 0

  for i in range(len(vetor1)):
    prod_interno += vetor1[i]*vetor2[i]

  norma_vetor1 = sum([x**2 for x in vetor1])**0.5
  norma_vetor2 = sum([x**2 for x in vetor2])**0.5

  return prod_interno/(norma_vetor1*norma_vetor2)

In [36]:
def vector(vec, idx, flag):
  if (flag == 0):
    return vec.iloc[idx]
  else:
    return vec[idx]

In [37]:
def text_retrieval(dataset, embedding, flag=1):
  top10 = [0] * 10

  for i in range(len(dataset)):
    best = []
    for j in range(len(dataset)):
      if i == j: continue
      best.append( (cosine_similarity(vector(embedding, j, flag), vector(embedding, i, flag)), j) )

    best.sort(reverse=True)
    top = best[:10]

    cat_orig = dataset.iloc[i]

    for idx, [sim, j] in enumerate(top):
      top10[idx] += cat_orig in dataset.iloc[j]

  return top10

## TFIDF

In [38]:
cuisine100 = df_cuisine['cuisine'][:100]
indian100 = df_indian['diet'][:100]

In [39]:
top10_tfidf_cuisine = text_retrieval(cuisine100, df_tfidf_cuisine, 0)
top10_tfidf_indian = text_retrieval(indian100, df_tfidf_indian, 0)

In [40]:
top10_tfidf_cuisine

[42, 30, 30, 23, 24, 22, 22, 18, 9, 17]

In [41]:
top10_tfidf_indian

[96, 96, 94, 93, 93, 94, 95, 95, 93, 95]

In [42]:
top10_word2vec_cuisine = text_retrieval(cuisine100, word2vec_cuisine)
top10_word2vec_indian = text_retrieval(indian100, word2vec_indian)

In [43]:
top10_word2vec_cuisine

[51, 46, 35, 34, 31, 28, 27, 16, 18, 20]

In [44]:
top10_word2vec_indian

[96, 94, 93, 94, 92, 92, 92, 92, 93, 95]

In [45]:
top10_bert_cuisine = text_retrieval(cuisine100, bert_cuisine)
top10_bert_indian = text_retrieval(indian100, bert_indian)

In [46]:
top10_bert_cuisine

[25, 18, 18, 21, 16, 13, 13, 12, 15, 4]

In [47]:
top10_bert_indian

[99, 96, 98, 96, 94, 93, 92, 93, 93, 95]